<a href="https://colab.research.google.com/github/Ha1ion/2025_NLP_HW2/blob/main/nlp_hw2_LSTM_3digit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

此作業有使用Gemini幫忙下註解

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install seaborn
! pip install opencc
! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns
import opencc
import os
from sklearn.model_selection import train_test_split

data_path = './data'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 96.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [ ]:
data_path = '/content/drive/MyDrive/2025_NLP_HW2'
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train_3digit.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,src,tgt
0,728*120=,87360
1,178-447=,-269
2,785*166=,130310
3,322-521=,-199
4,202+922=,1124


實驗二：訓練三位數，評估二位數

In [ ]:
import random
import pandas as pd
import os
from tqdm.notebook import tqdm

def generate_arithmetic_data_3digit_final(num_samples):
    """
    生成一個更符合實驗定義的訓練集：
    1. 確保算式中至少包含一個三位數數字。
    2. 混合多種題型，包括括號。
    """
    data = []
    operators = ['+', '-', '*']

    print(f"Generating {num_samples} complex 3-digit samples...")
    pbar = tqdm(range(num_samples))

    for _ in pbar:
        # 隨機決定要生成哪種題型
        # 'n3_op_n3': 3位數 op 3位數
        # 'n3_op_(n2_op_n1)': 3位數 op (2位數 op 1位數)
        # '(n2_op_n2)_op_n3': (2位數 op 2位數) op 3位數
        structure_type = random.choice(['n3_op_n3', 'n3_op_n3', 'n3_op_(n2_op_n1)', '(n2_op_n2)_op_n3'])

        try:
            if structure_type == 'n3_op_n3':
                n1 = random.randint(100, 999)
                n2 = random.randint(100, 999)
                op = random.choice(operators)
                expression = f"{n1}{op}{n2}"

            elif structure_type == 'n3_op_(n2_op_n1)':
                n1 = random.randint(100, 999)
                n2 = random.randint(10, 99)
                n3 = random.randint(1, 9)
                op1 = random.choice(['+', '-'])
                op2 = random.choice(operators)
                expression = f"{n1}{op2}({n2}{op1}{n3})"

            else: # '(n2_op_n2)_op_n3'
                n1 = random.randint(10, 99)
                n2 = random.randint(10, 99)
                n3 = random.randint(100, 999)
                op1 = random.choice(['+', '-'])
                op2 = random.choice(operators)
                expression = f"({n1}{op1}{n2}){op2}{n3}"

            answer = eval(expression)
            # 限制答案長度，避免過於極端的樣本
            if len(str(answer)) > 8:
                continue

            src = f"{expression}="
            tgt = str(answer)
            data.append({'src': src, 'tgt': tgt})
        except:
            continue

    return pd.DataFrame(data)

# --- 執行生成 ---
# 掛載雲端硬碟 (如果尚未掛載)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    data_path = '/content/drive/MyDrive/2025_NLP_HW2'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
except ImportError:
    data_path = '.' # 如果不是在 Colab 環境，則存在當前目錄

output_file = os.path.join(data_path, 'arithmetic_train_3digit_final.csv')

# 生成 26 萬筆，包含括號與三位數的訓練資料
df_train_3digit_final = generate_arithmetic_data_3digit_final(260000)
df_train_3digit_final.to_csv(output_file, index=False)

print(f"\nFile '{os.path.basename(output_file)}' has been generated.")
print("\nSample data:")
print(df_train_3digit_final.head())

Generating 3-digit training data...
File 'arithmetic_train_3digit.csv' has been generated.


In [ ]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))

20% wrong

In [ ]:
import pandas as pd
import numpy as np
import os

# --- 1. 設定您存放資料的路徑 ---
data_path = '/content/drive/MyDrive/2025_NLP_HW2'

# --- 2. 組合出完整的讀取與儲存路徑 ---
original_train_file = os.path.join(data_path, 'arithmetic_train.csv')
output_noisy_file = os.path.join(data_path, 'arithmetic_train_noisy.csv')

try:
    # --- 3. 讀取原始檔案 ---
    print(f"正在從 '{original_train_file}' 讀取資料...")
    df_train = pd.read_csv(original_train_file)
    print("讀取完成。")

    df_train_noisy = df_train.copy()

    # --- 4. 製造並寫入錯誤資料 (高效的向量化版本) ---
    print("正在生成 20% 的錯誤答案 (優化版)...")

    # 隨機選出 20% 的樣本索引
    noisy_indices = df_train_noisy.sample(frac=0.20, random_state=42).index

    # 將 'tgt' 欄位轉換為數值型別以進行計算
    original_answers = pd.to_numeric(df_train_noisy.loc[noisy_indices, 'tgt'], errors='coerce')

    # 過濾掉無法轉換的非數值資料
    valid_indices = original_answers.dropna().index

    # 一次性生成所有隨機偏移量
    num_to_modify = len(valid_indices)
    offsets = np.random.randint(1, 21, size=num_to_modify) * np.random.choice([-1, 1], size=num_to_modify)

    # 進行向量化計算，產生所有錯誤答案
    noisy_answers = original_answers.loc[valid_indices] + offsets

    # 將新的錯誤答案 (轉回字串格式) 一次性地寫回 DataFrame
    df_train_noisy.loc[valid_indices, 'tgt'] = noisy_answers.astype(str)

    print("錯誤答案生成完畢。")

    # 將包含錯誤答案的 DataFrame 儲存為新的 CSV 檔案
    print(f"正在將結果儲存至 '{output_noisy_file}'...")
    df_train_noisy.to_csv(output_noisy_file, index=False)

    print("\n" + "="*50)
    print(f"成功！新的訓練集檔案 'arithmetic_train_noisy.csv' 已經生成。")
    print("="*50)

except FileNotFoundError:
    print(f"錯誤：在 '{data_path}' 中找不到 'arithmetic_train.csv' 檔案。")
except Exception as e:
    print(f"程式執行時發生未預期的錯誤: {e}")

正在從 '/content/drive/MyDrive/2025_NLP_HW2/arithmetic_train.csv' 讀取資料...
讀取完成。
正在生成 20% 的錯誤答案 (優化版)...


/tmp/ipython-input-2944731329.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['-1' '-122' '-120' ... '2' '1272' '-11']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train_noisy.loc[valid_indices, 'tgt'] = noisy_answers.astype(str)


錯誤答案生成完畢。
正在將結果儲存至 '/content/drive/MyDrive/2025_NLP_HW2/arithmetic_train_noisy.csv'...

成功！新的訓練集檔案 'arithmetic_train_noisy.csv' 已經生成。


# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [ ]:
char_to_id = {}
id_to_char = {}

# --- ↓↓↓ 關鍵修正：同時從訓練集和評估集建立字庫 ↓↓↓ ---
vocab = set()
# 1. 加入訓練集的所有字元
for text in df_train['src']:
    vocab.update(list(text))
# 2. 加入評估集的所有字元
for text in df_eval['src']:
    vocab.update(list(text))
# --- ↑↑↑ 關鍵修正結束 ↑↑↑ ---

# 特殊 tokens
special_tokens = ['<pad>', '<eos>']

# 建立 char_to_id 和 id_to_char 字典
char_to_id = {token: i for i, token in enumerate(special_tokens + sorted(list(vocab)))}
id_to_char = {i: token for token, i in char_to_id.items()}

vocab_size = len(char_to_id)
print(f'Vocab size: {vocab_size}')
print(f'Dictionary content: {char_to_id.keys()}')

Vocab size: 18
Dictionary content: dict_keys(['<pad>', '<eos>', '(', ')', '*', '+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '='])


# Data Preprocessing
 - The data is processed into the format required for the model's input and output. (End with \<eos\> token)


In [ ]:
def text_to_ids(text, char_map):
    return [char_map[char] for char in text]


#df_train['src'] = df_train['src'].str.replace('=', '') + '=' + df_train['tgt']

df_train['char_id_list'] = df_train['src'].apply(lambda x: text_to_ids(x, char_to_id) + [char_to_id['<eos>']])

def create_shifted_label(char_ids):
    return char_ids[1:] + [char_to_id['<pad>']]

df_train['label_id_list'] = df_train['char_id_list'].apply(create_shifted_label)

df_eval['src'] = df_eval['src'].str.replace('=', '') + '='

df_train.head()

,src,tgt,char_id_list,label_id_list,len
0,728*120=87360,87360,"[14, 9, 15, 4, 8, 9, 7, 17, 15, 14, 10, 13, 7, 1]","[9, 15, 4, 8, 9, 7, 17, 15, 14, 10, 13, 7, 1, 0]",13
1,178-447=-269,-269,"[8, 14, 15, 6, 11, 11, 14, 17, 6, 9, 13, 16, 1]","[14, 15, 6, 11, 11, 14, 17, 6, 9, 13, 16, 1, 0]",12
2,785*166=130310,130310,"[14, 15, 12, 4, 8, 13, 13, 17, 8, 10, 7, 10, 8...","[15, 12, 4, 8, 13, 13, 17, 8, 10, 7, 10, 8, 7,...",14
3,322-521=-199,-199,"[10, 9, 9, 6, 12, 9, 8, 17, 6, 8, 16, 16, 1]","[9, 9, 6, 12, 9, 8, 17, 6, 8, 16, 16, 1, 0]",12
4,202+922=1124,1124,"[9, 7, 9, 5, 16, 9, 9, 17, 8, 8, 9, 11, 1]","[7, 9, 5, 16, 9, 9, 17, 8, 8, 9, 11, 1, 0]",12


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [ ]:
batch_size = 64
epochs = 2
embed_dim = 512
hidden_dim = 512
lr = 0.0005
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

- Example: 1+2-3=0
    - Model input: 1 + 2 - 3 = 0
    - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
    - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        # return the amount of data
        return len(self.sequences) # Write your code here

    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        row = self.sequences.iloc[index]
        x = row['char_id_list'] # Write your code here
        y = row['label_id_list'] # Write your code here
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])

    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])

    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])

    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [ ]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']])

In [ ]:
# Build dataloader of train set and eval set, collate_fn is the collate function
ds_train = Dataset(df_train) # Write your code here

dl_train = torch.utils.data.DataLoader(ds_train,
                                       batch_size=batch_size,
                                       shuffle=True,
                                       collate_fn=collate_fn)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [ ]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()

        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])

        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)

        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))

    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)

    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)

        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)

        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)

        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)

        batch_x = self.linear(batch_x)

        return batch_x

    def generator(self, start_char, max_len=200):

        char_list = [char_to_id[c] for c in start_char]

        next_char = None

        while len(char_list) < max_len:
            # Write your code here
            # Pack the char_list to tensor
            # Input the tensor to the embedding layer, LSTM layers, linear respectively

            input_seq = torch.LongTensor([char_list]).to(next(self.parameters()).device)
            y = self.encoder(input_seq, torch.LongTensor([len(char_list)])) # Obtain the next token prediction y

            last_time_step_pred = y[0, -1, :]

            next_char_id = torch.argmax(last_time_step_pred).item()

            next_char = next_char_id # Use argmax function to get the next token prediction

            if next_char == char_to_id['<eos>']:
                break

            char_list.append(next_char)

        return [id_to_char[ch_id] for ch_id in char_list]

In [ ]:
torch.manual_seed(2)

if not torch.cuda.is_available():
    raise RuntimeError("GPU not available. Please change the runtime type to GPU.")
device = torch.device('cuda')
print(f"Device set to: {device}")

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

model.to(device)

Device set to: cuda


CharRNN(
  (embedding): Embedding(18, 512, padding_idx=0)
  (rnn_layer1): LSTM(512, 512, batch_first=True)
  (rnn_layer2): LSTM(512, 512, batch_first=True)
  (linear): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=18, bias=True)
  )
)

In [ ]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
# Write your code here. Cross-entropy loss function. The loss function should ignore <pad>
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)# Write your code here. Use Adam or AdamW for Optimizer

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [ ]:
from tqdm import tqdm
from copy import deepcopy
model = model.to(device)
i = 0

# --- 外層主迴圈 ---
for epoch in range(1, epochs + 1):
    # --- 1. 訓練階段 ---
    model.train()
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        optimizer.zero_grad()
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip)
        optimizer.step()
        bar.set_postfix(loss=loss.item())

    # --- 2. 評估階段 ---
    model.eval()
    matched = 0
    total = 0
    bar_eval = tqdm(df_eval.iterrows(), desc=f"Validation epoch {epoch}")

    with torch.no_grad(): # 在評估時加入 no_grad() 是個好習慣
        for _, row in bar_eval:
            batch_x = row['src']
            batch_y = row['tgt']

            prediction_chars = model.generator(batch_x)
            prediction_str = "".join(prediction_chars)

            if '=' in prediction_str:
                answer_part = prediction_str.split('=', 1)[1]
                predicted_answer = answer_part.split('<eos>', 1)[0]

                # ---  ↓↓↓ 最關鍵的修正 ↓↓↓ ---
                # 在比對前，去除答案前後可能存在的空白字元
                predicted_answer = predicted_answer.strip()
                # ---  ↑↑↑ 最關鍵的修正 ↑↑↑ ---

            else:
                predicted_answer = ""

            if predicted_answer == batch_y:
                matched += 1
            total += 1

    # --- 3. 印出該 epoch 的準確率 ---
    if total > 0:
        accuracy = matched / total
        print(f"Epoch {epoch} Validation Accuracy: {accuracy:.4f}")
    else:
        print(f"Epoch {epoch} Validation: No data evaluated.")

Train epoch 1: 100%|██████████| 4063/4063 [01:12<00:00, 55.79it/s, loss=1.35]
Validation epoch 1: 263250it [28:37, 153.25it/s]


Epoch 1 Validation Accuracy: 0.0018


Train epoch 2: 100%|██████████| 4063/4063 [01:12<00:00, 56.27it/s, loss=1.26]
Validation epoch 2: 2734it [00:18, 146.87it/s]


KeyboardInterrupt: 